In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

# Introduction

[Yelp.com](www.yelp.com) is a popular crowd-sourced local business review and social network site. It offers a platform where users can submit a review of the business using a one to five star rating system along with additional reviews. In this project, we are interested in the sentiment of reviews and trying to explore what could be potential useful factors to predict the sentiment. 

# Data Overview


All the data is retrieved using the Yelp Fusion API. We search for restaurants in 10 cities in the United States, namely Boston, Chicago, Miami, Minneapolis, Nashville, New Mexico, New York, Pittsburgh, San Francisco and Seattle. 

For each city, we collect at most 1,000 restaurants and at most 3 reviews (due to the API limitation) for each restaurant. There are in total 9915 restaurants and 29742 reviews. In additional to the review text and the review rating, we also retrieved other business-related information including the city location, overall  restaurant rating, category, price level and the total review count for each restaurant. Data are then preprocessed and stored in a dataframe. 

In [3]:
# load data into a panda dataframe
data = pd.read_pickle('complete_data')

# show first 2 row of the data
data.head(2)

review_id           restaurant_id rating  \
0  MZRCnkRjWwilSI1tazv_3w  y2w6rFaO0XEiG5mFfOsiFA      5   
1  D7rxX5QE1uHJ1rcHCRjRkA  y2w6rFaO0XEiG5mFfOsiFA      5   

                                                text    city         category  \
0  The best lobster rolls in Boston. They give ve...  Boston  (Seafood, Bars)   
1  Amazing selection of oysters!! Hot lobster rol...  Boston  (Seafood, Bars)   

   restaurant_rating price review_count  
0                4.5   $$$         4041  
1                4.5   $$$         4041

# Exploratory Data Analysis


To begin with, we first perform a exploratory data analysis to get a through understading of our data and explore the relationship between each variables. 

We choose price, restaurant_rating, review_count and estimate (the estimated category) as explanatory variables, and rating as the response. In this section we will focus on the first three. 

In [4]:
# get categories
print("Price value:" + str(set(pd.Series.tolist(data['price']))))
print("Restaurant Rating: " + str(set(pd.Series.tolist(data['restaurant_rating']))))
print("Review Rating: " + str(set(pd.Series.tolist(data['rating']))))

Price value:{'$$', '$$$', '$', '$$$$'}
Restaurant Rating: {1.5, 2.5, 3.0, 3.5, 5.0, 4.5, 4.0, 2.0}
Review Rating: {1, 2, 3, 4, 5}


From the above outputs we see that price is a categorical variable with four levels: \$, \$\$, \$\$\$, \$\$\$\$. Restaurant_rating is also categorical with 8 categories: 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0. Review count is quantitative and takes values in positive integers. Then we will explore how each variable interact with the review rating.

# Statistical Analysis


We want to analyze the relationship between each of the three explanatory variables and the response. Since price and restaurant_rating are categorical, we will run chi-square test on them. 

In [5]:
# price ~ rating test
# get counts of combinations of different levels of price and rating
d = data.groupby(['price', 'rating']).size().reset_index(name='count')
p = pd.Series.tolist(d['price'])
prices = set(p)
r = pd.Series.tolist(d['rating'])
ratings = set(r)
c = pd.Series.tolist(d['count'])
l = [(p[i], r[i], c[i], 0) for i in range(len(p))]
for price in prices:
    for rating in ratings:
        # compute row total
        row = sum([z for (x, y, z, v) in l if y == rating])
        # compute column total
        col = sum([z for (x, y, z, v) in l if x == price])
        tot = sum([z for (x, y, z, v) in l])
        # compute expected for this entry
        exp = row*col/tot
        l1 = [(x, y, z, exp) for (x, y, z, v) in l if (x == price and y == rating)]
        l2 = [(x, y, z, v) for (x, y, z, v) in l if (x != price or y != rating)]
        l = l1+l2
# sum up (observed - expected)^2/expected
chisq = np.sum([(z-v)**2/v for (x, y, z, v) in l])
print('price ~ rating: ')
print('chi-squared stat: ' + str(chisq))
crit = stats.chi2.ppf(q = 0.95, df=(len(prices)-1)*(len(ratings)-1))
print('critical value: ' + str(crit))

price ~ rating: 
chi-squared stat: 147.38530044
critical value: 21.0260698175


Since chi-squared stat > critical value, we reject the null hypothesis and state there's a relationship between price and rating. 

#### Price - Rating Analysis 
We would like to analyze the relationship between price and review rating. From the visualization below, it turns out to be that the higher the resraurant price level is, the higher rating of each review for the restaurant. The resraurants in price \$$$$ level have larger proportion of 5 star review rating (more than half) and less propotion of 1 star review rating (nearly 5%) than others.
Also one thing can be inferred from this graph is that customers are more likely to give high ratings no matter what price level the resraurant is.

So that we can infer that the luxury restaurant tends to have higher review ratings and cheap restaurants also have this feature. But cheap restaurants have the largest propotion of 1 star ratings. While, the moderately priced restaurant have the relatively normal distribution. 

# Summary

In this project, we are interested in the sentiment of the reviews on Yelp. we first did data collection which includes ten cities and 3 reviews for each cities using the Yelp API. After that, we did data pre-processing to merge the raw data into a dataframe where each entry include the review rating, review text, restaurant rating and restaurant price level and etc.

Then we did some explantory and statistical analysis to get a brief understanding of the data. Next we built a naive bayes classifier using only review text that achieves around 71% prediction sentiment accuracy. By incorporating the results (predicted categories) from the NB classifier into a logistic regression model with other explanatory variables, the prediction accuracy improves to around 74%. 